This section focusses on combining two or more DataFrames together. Connecting DataFrames based on common values across one or more columns. 

Topics covered in this section:
- The pd.concat() method.
- Inner joins.
- Outer joins.
- Left joins. 
- The left_on and right_on parameters.
- Merging by indexes with the left_index and right_index parameters. 
- The .join() method. 

In [121]:
import pandas as pd
pd.__version__

'1.1.3'

The .pd_concat() Method - Part 1 of 2

Concatenates two or more DataFrames into a new into. Takes the rows of one DataFrame and adds it to the rows of another DataFrame to obtain a brand new DataFrame object.  

In [8]:
week1 = pd.read_csv("Restaurant - Week 1 Sales.csv") # Which customer came in and what they bought (can be multiple times). 
week2 = pd.read_csv("Restaurant - Week 2 Sales.csv")
customers = pd.read_csv("Restaurant - Customers.csv") # List of customers. 
foods = pd.read_csv("Restaurant - Foods.csv") # Restaurant menu. 

In [10]:
week1.head(3) # Columns: Customer ID, Food ID.
week2.head(3) # Columns: Customer ID, Food ID.
customers.head(3) # Columns: ID, First Name. Last Name, Gender, Company, Occupation, 
foods.head(3) # Columns: Food ID, Food Item, Price. 

,Food ID,Food Item,Price
0,1,Sushi,3.99
1,2,Burrito,9.99
2,3,Taco,2.99


In [13]:
# .pd.concat() method.
# Parameters:
# obs : provide list of the DataFrames you want to concatenate. 
# ignore_index : by default = False. Set to True to drop existing indexes for  DataFrames and generate a brand new one from scratch. 

pd.concat(objs = [week1, week2])

# There are 250 rows in each DataFrame, therefore after concatenation, there will be 500 rows in total. 
# The order of the DataFrame matters. week1 will be at the top. 
# By default, ignore_index = False, Pandas

,Customer ID,Food ID
0,537,9
1,97,4
2,658,1
3,202,2
4,155,9
...,...,...
245,783,10
246,556,10
247,547,9
248,252,9


In [15]:
pd.concat(objs = [week1, week2], ignore_index = True)

# Returns concatenated DataFrame with new index, in this case 0-499 ( instead of 0 - 249 repeated). 

,Customer ID,Food ID
0,537,9
1,97,4
2,658,1
3,202,2
4,155,9
...,...,...
495,783,10
496,556,10
497,547,9
498,252,9


In [16]:
# .append() method : an alternative approach to concatenate DataFrames.

week1.append(week2)

# Returns conatenated DataFrame of week1 and week2 with the original indexes of both DataFrames preserved. 
# The ignore_ignore parameter is available for .append() too. 

,Customer ID,Food ID
0,537,9
1,97,4
2,658,1
3,202,2
4,155,9
...,...,...
245,783,10
246,556,10
247,547,9
248,252,9


The .pdconcat() Method - Part 2 of 2

In [17]:
week1 = pd.read_csv("Restaurant - Week 1 Sales.csv") # Which customer came in and what they bought (can be multiple times). 
week2 = pd.read_csv("Restaurant - Week 2 Sales.csv")
customers = pd.read_csv("Restaurant - Customers.csv") # List of customers. 
foods = pd.read_csv("Restaurant - Foods.csv") # Restaurant menu. 

In [19]:
# We want a unique identifier for each row but at the same time do not want to lose the original index. 
# We can do this by using MultiIndex DataFrames and by adding keys before each index position then we can identify which 
# DataFrame the row came from. 

sales = pd.concat(objs = [week1, week2], keys = ["Week 1", "Week2"])
sales.head(3)

# We haven't reset the index of the week1 and week2 DataFrames, but we nested them within a respective level of a MultiIndex.
# Now, each row has a unique identifier and we also retain the logic of where each row came from in our original DataFrame. 

# keys list must be of the same length of the objs list. 
# keys are the index labels of the outer-most index level of the MultiIndex DataFrame. 

Customer ID  Food ID
Week 1 0          537        9
       1           97        4
       2          658        1

In [25]:
# We can now access specific rows in our data using .loc[]. 

sales.loc[("Week 1", )] # Returns just the rows from week1. 
sales.loc[("Week 1", 240)] # Returns row with index "Week 1" in first level and index 240 in second index level. 

sales.loc[("Week 1", 240), "Customer ID"]
# Returns the Customer ID value for the row with MultiIndex labels Week 1 and 240. 

Customer ID    945
Food ID          5
Name: (Week 1, 240), dtype: int64

Inner Joins - Part 1 of 2

Inner joins merges datasets based on commonly shared values using the .merge() method. 

Equivalent of the intersection (A AND B). 

In [26]:
week1 = pd.read_csv("Restaurant - Week 1 Sales.csv") # Which customer came in and what they bought (can be multiple times). 
week2 = pd.read_csv("Restaurant - Week 2 Sales.csv")
customers = pd.read_csv("Restaurant - Customers.csv") # List of customers. 
foods = pd.read_csv("Restaurant - Foods.csv") # Restaurant menu. 

In [31]:
# .merge() parameters:
# right : the right DataFrame. the first DataFrame (week1) is the left DataFrame.
# how : left, right, inner, outer etc.
# on : the column you want to merge on (must be present in both DataFrames). 
# suffixes : provide list of the suffixes you want to add to the duplicated columns. 

week1.merge(right = week2, how = 'inner', on = "Customer ID").head(4)

# Returns a merged DataFrame on the Customer ID column. 
# Food ID_x and Food ID_y is the Food ID column for week 1 and 2 respectively. The reason for the suffices is because it has to merge 
# the values from both of the DataFrames whenever it finds a common Customer ID but it doesn't want duplicate column names. 

,Customer ID,Food ID_x,Food ID_y
0,537,9,5
1,155,9,3
2,155,1,3
3,503,5,8


In [35]:
# Use a filtration process to extract the values from a Customer ID column from each DataFrame where the Customer ID Iis 155.

week1[week1["Customer ID"] == 155]
# There's 2 occurrences of customer id 155 in week 1. Basically, this person came in twice in week 1 and got food item 9 and 1. 

,Customer ID,Food ID
4,155,9
17,155,1


In [34]:
week2[week2["Customer ID"] == 155]

# In second week, this customer came in once and got food item 3. 

# But since there were 2 occurrences in week 1, the merged dataset will have a repeat of the food ID 3 from week 2. 

,Customer ID,Food ID
208,155,3


Inner Joins - Part 2 of 2

In [36]:
week1 = pd.read_csv("Restaurant - Week 1 Sales.csv") # Which customer came in and what they bought (can be multiple times). 
week2 = pd.read_csv("Restaurant - Week 2 Sales.csv")
customers = pd.read_csv("Restaurant - Customers.csv") # List of customers. 
foods = pd.read_csv("Restaurant - Foods.csv") # Restaurant menu. 

In [37]:
# Pull in a new DataFrame of those customers who came in multiple weeks (both week 1 and week 2) and also order the exact 
# same food item on both week 1 and week 2. 

week1.merge(right = week2, how = "inner", on = ["Customer ID", "Food ID"])

# Looks for matches in both Customer ID and Food ID, both of these conditions have to be satisfied. 
# Only the rows where both the Customer ID and Food ID match are merged. 

# Since both column are going to be the same, there is no need for duplicated columns with suffixesm. 

,Customer ID,Food ID
0,304,3
1,540,3
2,937,10
3,233,3
4,21,4
5,21,4
6,922,1
7,578,5
8,578,5


In [39]:
# Taking a closer look at the duplicate rows (index label 4 and 5).
week1[week1["Customer ID"] == 21]

# This customer came in and bought the same item twice in week 1.

,Customer ID,Food ID
101,21,4
212,21,4


In [41]:
week2[week2["Customer ID"] == 21]

# Came in once in week 2 and bought the same item as in week 1 .

# Process
# When Python is running through the rows in week 1 and finds the first 21, 4 pair, it will pair it with the match 21, 4 in week 2.
# Then when it moves onto the second 21,1 in week 1, it will match it again with the 21, 4 in week 1 since it matches. 

# If there were two of the same Customer ID and Food ID combinations in both week 1 and week 2, then Python will criss-match
# match these four pairs and end up with four matches (1a, 2a), (1a, 2b), (1b, 2a), (1b, 2b). 

,Customer ID,Food ID
30,21,4


Outer Joins

Two types of outer joins:
- Either A OR B. - Full Outer Join. 
- Either A OR B but not A AND B. 

In [42]:
week1 = pd.read_csv("Restaurant - Week 1 Sales.csv") # Which customer came in and what they bought (can be multiple times). 
week2 = pd.read_csv("Restaurant - Week 2 Sales.csv")
customers = pd.read_csv("Restaurant - Customers.csv") # List of customers. 
foods = pd.read_csv("Restaurant - Foods.csv") # Restaurant menu. 

In [47]:
week1.merge(right = week2, how = "outer", on = "Customer ID", suffixes = [" - Week 1", " - Week 2"])

# For rows with no NULL values, that means that Customer ID was found in both week 1 and week 2. 
# NULL values in week 1 means that the customer only came in on week 2.

# Since we are combining rows which have common Customer IDs, this DataFrame will be shorter than if we used concatenation. 
# Using the concatenation method, the DataFrame will have 500 rows (250 rows each for week 1 and 2), since we are just
# appending week 1 on to week 2. 

# Using .merge() method will still have all the customer IDs represented, but those that match will not just be placed on top of 
# each other but merged on the common line, whereas ones without a match will have its separate line with a NULL value for one
# of the columns. 

,Customer ID,Food ID - Week 1,Food ID - Week 2
0,537,9.0,5.0
1,97,4.0,NaN
2,658,1.0,NaN
3,202,2.0,NaN
4,155,9.0,3.0
...,...,...,...
449,855,NaN,4.0
450,559,NaN,10.0
451,276,NaN,4.0
452,556,NaN,10.0


In [48]:
len(week1.merge(right = week2, how = "outer", on = "Customer ID", suffixes = [" - Week 1", " - Week 2"]))

# In total, if simply concatenated, there will be 500 rows. By performing an outer join, we have 454. 
# This means that 46 rows were able to match the Customer IDs together from week 1 and week 2. The inner component of the join
# has 46 rows (week 1 AND week 2). 
# Whenever it merges, it's able to cut out an extra row (since both rows are combined), whenever it is unable to merge, it has to 
# keep that row and apply a NULL value for the other column. 

454

In [52]:
# Parameter 
# Indicator= True : Adds an additional column on the right 
#                             side of the DataFrame and tells you which table the value is pulled from. 

merged = week1.merge(right = week2, how = "outer", on = "Customer ID", indicator = True)
merged # Returns DataFrame with the additional _merge column showing where the value is pulled from. 

merged["_merge"].value_counts()
# Returns a value count to show how the values are distributed. 

right_only    197
left_only     195
both           62
Name: _merge, dtype: int64

In [67]:
# A OR B but not A & B: We can also use this to cut out values that are from both week 1 or week 2. And end up with left or right only. 

left_or_right = merged["_merge"].isin(["left_only", "right_only"]) # Exists in either but not both. 
merged[left_or_right] 
# Returns DataFrame consisting of row which are in either week 1 or week 2 but not both. 

# or equivalently
# merged[~(merged["_merge"] == "both")]

,Customer ID,Food ID_x,Food ID_y,_merge
1,97,4.0,NaN,left_only
2,658,1.0,NaN,left_only
3,202,2.0,NaN,left_only
6,213,8.0,NaN,left_only
7,600,1.0,NaN,left_only
...,...,...,...,...
449,855,NaN,4.0,right_only
450,559,NaN,10.0,right_only
451,276,NaN,4.0,right_only
452,556,NaN,10.0,right_only


Left Joins

Pulls values that exist in the left table. If they also exist in the right table then pull that value too, but if it doesn't then returns a NULL value. Values that are exclusively in the right table will not be pulled. Right join is the same thing but in reverse. 

In [ ]:
week1 = pd.read_csv("Restaurant - Week 1 Sales.csv") # Which customer came in and what they bought (can be multiple times). 
week2 = pd.read_csv("Restaurant - Week 2 Sales.csv")
customers = pd.read_csv("Restaurant - Customers.csv") # List of customers. 
foods = pd.read_csv("Restaurant - Foods.csv") # Restaurant menu. 

In [69]:
week1.head(3)

,Customer ID,Food ID
0,537,9
1,97,4
2,658,1


In [70]:
foods.head(3)

,Food ID,Food Item,Price
0,1,Sushi,3.99
1,2,Burrito,9.99
2,3,Taco,2.99


In [74]:
week1.merge(right = foods, how = "left", on = "Food ID", sort = True)

# sort : sorts the matching column. 

,Customer ID,Food ID,Food Item,Price
0,658,1,Sushi,3.99
1,600,1,Sushi,3.99
2,155,1,Sushi,3.99
3,341,1,Sushi,3.99
4,20,1,Sushi,3.99
...,...,...,...,...
245,809,10,Drink,1.75
246,584,10,Drink,1.75
247,274,10,Drink,1.75
248,151,10,Drink,1.75


The left_on and right_on Parameters

In previous lessons, the column we wanted to join from the left and right DataFrame had the same name. 

The left_on and right_on parameters allow us to join a column which has different names in the left and right table. 

In [75]:
week1 = pd.read_csv("Restaurant - Week 1 Sales.csv") # Which customer came in and what they bought (can be multiple times). 
week2 = pd.read_csv("Restaurant - Week 2 Sales.csv")
customers = pd.read_csv("Restaurant - Customers.csv") # List of customers. 
foods = pd.read_csv("Restaurant - Foods.csv") # Restaurant menu. 

In [76]:
week2.head(3)

,Customer ID,Food ID
0,688,10
1,813,7
2,495,10


In [78]:
customers.head(3)

# For the column we want to join, the column name is Customer ID in the left table whereas in the right table it is ID. 

,ID,First Name,Last Name,Gender,Company,Occupation
0,1,Joseph,Perkins,Male,Dynazzy,Community Outreach Specialist
1,2,Jennifer,Alvarez,Female,DabZ,Senior Quality Engineer
2,3,Roger,Black,Male,Tagfeed,Account Executive


In [86]:
# Joining using the left_on and right_on parameter.

week2.merge(right = customers, how = "left", left_on = "Customer ID", right_on = "ID", sort = True)

# Note that since the joining column has different names, both columns (Customer ID and ID) have been included in the DataFrame.
# We can see that it is merged correctly since values for Customer ID and ID is the same, however, one column is not needed. 

,Customer ID,Food ID,ID,First Name,Last Name,Gender,Company,Occupation
0,8,6,8,Frances,Adams,Female,Dabshots,Developer III
1,13,2,13,Ruth,Alvarez,Female,Twitterlist,Mechanical Systems Engineer
2,21,4,21,Albert,Burns,Male,Rhynoodle,Junior Executive
3,24,8,24,Donna,Thomas,Female,Jaxbean,Chief Design Engineer
4,27,4,27,Jessica,Bennett,Female,Twitternation,Account Executive
...,...,...,...,...,...,...,...,...
245,968,4,968,Teresa,Reynolds,Female,Flashdog,Budget/Accounting Analyst IV
246,969,3,969,Marie,Ortiz,Female,Meejo,Research Associate
247,977,7,977,Cynthia,Dixon,Female,Skalith,Automation Specialist II
248,994,2,994,Clarence,Morgan,Male,Edgewire,Geologist II


In [87]:
# .drop() : we can use this method to specify which column to drop.

# Since Column ID and ID are duplicated columns, we will drop the ID column. 

week2 = week2.merge(right = customers, how = "left", left_on = "Customer ID", right_on = "ID", sort = True).drop("ID", axis = "columns")
week2  # Assign if you want to overwrite the DataFrame.

,Customer ID,Food ID,First Name,Last Name,Gender,Company,Occupation
0,8,6,Frances,Adams,Female,Dabshots,Developer III
1,13,2,Ruth,Alvarez,Female,Twitterlist,Mechanical Systems Engineer
2,21,4,Albert,Burns,Male,Rhynoodle,Junior Executive
3,24,8,Donna,Thomas,Female,Jaxbean,Chief Design Engineer
4,27,4,Jessica,Bennett,Female,Twitternation,Account Executive
...,...,...,...,...,...,...,...
245,968,4,Teresa,Reynolds,Female,Flashdog,Budget/Accounting Analyst IV
246,969,3,Marie,Ortiz,Female,Meejo,Research Associate
247,977,7,Cynthia,Dixon,Female,Skalith,Automation Specialist II
248,994,2,Clarence,Morgan,Male,Edgewire,Geologist II


Merging by Indexes with the left_index and right_index Parameters

Previously, we were merging on column values, now we will learn how to merge by index values. This may be the case of merging based on common indexes, e.g. take the left index and take the right index and merge them together. Or we may want to merge the index values in the left DataFrame with the column values in the right DataFrame. 

In [91]:
week1 = pd.read_csv("Restaurant - Week 1 Sales.csv") # Which customer came in and what they bought (can be multiple times). 
week2 = pd.read_csv("Restaurant - Week 2 Sales.csv")
customers = pd.read_csv("Restaurant - Customers.csv", index_col = "ID") # List of customers. 
foods = pd.read_csv("Restaurant - Foods.csv", index_col = "Food ID") # Restaurant menu. 

In [92]:
customers.head(2)

,First Name,Last Name,Gender,Company,Occupation
ID,,,,,
1,Joseph,Perkins,Male,Dynazzy,Community Outreach Specialist
2,Jennifer,Alvarez,Female,DabZ,Senior Quality Engineer


In [94]:
foods.head(2)

,Food Item,Price
Food ID,,
1,Sushi,3.99
2,Burrito,9.99


In [95]:
week1.head(2)

,Customer ID,Food ID
0,537,9
1,97,4


In [105]:
# Merge the customers and foods DataFrames with week 1. 

# First, merge week1 with customers. 

sales = week1.merge(right = customers, how = "left", left_on = "Customer ID", right_index = True)
sales
# Note: we need to merge the left column (Customer ID) with the index from the right table (ID). We set right_index = True, which
# by default is = False and now, it will match the values in the Customer ID column from the left DataFrame (week1) with the index
# labels from the right table (customers)

# One advantage of using indexes to merge is that those duplicate columns do not arise. The Customer ID column appears but the 
# index label was not brought in. 


,Customer ID,Food ID,First Name,Last Name,Gender,Company,Occupation
0,537,9,Cheryl,Carroll,Female,Zoombeat,Registered Nurse
1,97,4,Amanda,Watkins,Female,Ozu,Account Coordinator
2,658,1,Patrick,Webb,Male,Browsebug,Community Outreach Specialist
3,202,2,Louis,Campbell,Male,Rhynoodle,Account Representative III
4,155,9,Carolyn,Diaz,Female,Gigazoom,Database Administrator III
...,...,...,...,...,...,...,...
245,413,9,Diane,Bailey,Female,Wikibox,Technical Writer
246,926,6,Anne,Wagner,Female,Skyba,Legal Assistant
247,134,3,Diana,Hall,Female,Quinu,Financial Advisor
248,396,6,Juan,Romero,Male,Zoonder,Analyst Programmer


In [109]:
# Now we want to merge an additional foods DataFrame to the newly created sales DataFrame. 
sales = sales.merge(right = foods, how = "left", left_on = "Food ID", right_index = True)
sales.head(3)

# We reassigned the sales DataFrame again to this final twice merged DataFrame. 

,Customer ID,Food ID,First Name,Last Name,Gender,Company,Occupation,Food Item,Price
0,537,9,Cheryl,Carroll,Female,Zoombeat,Registered Nurse,Donut,0.99
1,97,4,Amanda,Watkins,Female,Ozu,Account Coordinator,Quesadilla,4.25
2,658,1,Patrick,Webb,Male,Browsebug,Community Outreach Specialist,Sushi,3.99


In [111]:
# Example 2: Merging by two indexes. 
# Assume that the rows are according to the time they came in. We want to match the customers of week 1 and week 2 based 
# on the order that they came in for their corresponding week. So, we merge by index. 

week1.head(2)

,Customer ID,Food ID
0,537,9
1,97,4


In [112]:
week2.head(2)

,Customer ID,Food ID
0,688,10
1,813,7


In [114]:
week1.merge(right = week2, how = "left", left_index = True, right_index = True, suffixes = (" - wk1", " - wk2"))

# We can see who came in first each week and what they ordered. 

# Pandas is very versatile. You can match by columns, indexes or combination of both. 

,Customer ID - wk1,Food ID - wk1,Customer ID - wk2,Food ID - wk2
0,537,9,688,10
1,97,4,813,7
2,658,1,495,10
3,202,2,189,5
4,155,9,267,3
...,...,...,...,...
245,413,9,783,10
246,926,6,556,10
247,134,3,547,9
248,396,6,252,9


The .join() Method

Simple method that can concatenate vertically when the DataFrames share the exact same index. You can think of it similar to an append or a concatenation process but on the other axis. 

This method is usually used when two datasets are linked together but come from different sources or have been extracted and separated by nature and you need to combine them. 

In [122]:
week1 = pd.read_csv("Restaurant - Week 1 Sales.csv") # Which customer came in and what they bought (can be multiple times). 
week2 = pd.read_csv("Restaurant - Week 2 Sales.csv")
customers = pd.read_csv("Restaurant - Customers.csv") # List of customers. 
foods = pd.read_csv("Restaurant - Foods.csv") # Restaurant menu. 
satisfaction = pd.read_csv("Restaurant - Week 1 Satisfaction.csv") # Satisfaction rating of customers in week1. 

In [118]:
week1.head(2)

,Customer ID,Food ID
0,537,9
1,97,4


In [120]:
satisfaction.head(2)

,Satisfaction Rating
0,2
1,7


In [123]:
# In this case, one way to join both tables is to use the merge method. 

week1.merge(right = satisfaction, how = "left", left_index = True, right_index = True)

,Customer ID,Food ID,Satisfaction Rating
0,537,9,2
1,97,4,7
2,658,1,3
3,202,2,7
4,155,9,10
...,...,...,...
245,413,9,1
246,926,6,2
247,134,3,8
248,396,6,10


In [126]:
# .join() method is more efficient way to achieve same results. 
# Can be used when two DataFrames have the exact same index. You just add the right table and add it to the right side of the 
# left table. 

week1.join(satisfaction).head()

# Achieves same result as if we used .merge(). In this case .join() is more efficient, however, .merge() is more versatile. 

,Customer ID,Food ID,Satisfaction Rating
0,537,9,2
1,97,4,7
2,658,1,3
3,202,2,7
4,155,9,10


The pd.merge() Method

Introduce .merge() method called directly on Pandas. This has the same function as before, just an alternate way of calling it. 

In [127]:
week1 = pd.read_csv("Restaurant - Week 1 Sales.csv") # Which customer came in and what they bought (can be multiple times). 
week2 = pd.read_csv("Restaurant - Week 2 Sales.csv")
customers = pd.read_csv("Restaurant - Customers.csv") # List of customers. 
foods = pd.read_csv("Restaurant - Foods.csv") # Restaurant menu. 
satisfaction = pd.read_csv("Restaurant - Week 1 Satisfaction.csv") # Satisfaction rating of customers in week1. 

In [129]:
week1.head(3)

,Customer ID,Food ID
0,537,9
1,97,4
2,658,1


In [130]:
customers.head(3)

,ID,First Name,Last Name,Gender,Company,Occupation
0,1,Joseph,Perkins,Male,Dynazzy,Community Outreach Specialist
1,2,Jennifer,Alvarez,Female,DabZ,Senior Quality Engineer
2,3,Roger,Black,Male,Tagfeed,Account Executive


In [132]:
pd.merge(left = week1, right = customers, how = "left", left_on = "Customer ID", right_on = "ID")

# week1.merge(right = customers, how = "left", left_on = "Customer ID", right_on = "ID") # Same function. 

,Customer ID,Food ID,ID,First Name,Last Name,Gender,Company,Occupation
0,537,9,537,Cheryl,Carroll,Female,Zoombeat,Registered Nurse
1,97,4,97,Amanda,Watkins,Female,Ozu,Account Coordinator
2,658,1,658,Patrick,Webb,Male,Browsebug,Community Outreach Specialist
3,202,2,202,Louis,Campbell,Male,Rhynoodle,Account Representative III
4,155,9,155,Carolyn,Diaz,Female,Gigazoom,Database Administrator III
...,...,...,...,...,...,...,...,...
245,413,9,413,Diane,Bailey,Female,Wikibox,Technical Writer
246,926,6,926,Anne,Wagner,Female,Skyba,Legal Assistant
247,134,3,134,Diana,Hall,Female,Quinu,Financial Advisor
248,396,6,396,Juan,Romero,Male,Zoonder,Analyst Programmer
